In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import re
plt.style.use('ggplot')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/Doceree-HCP_Train (2).csv
/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/DOCREE_SAMPLE_SUBMISSION.csv
/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/Doceree-HCP_Test.csv


In [2]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import csv

import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=814da08cedf5c8c7d083184d77afd93e4ca51205daae3279c6debbbd431c03a6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
#import cuml

In [4]:
train_path = "/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/Doceree-HCP_Train (2).csv"
test_path = "/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/Doceree-HCP_Test.csv"
sub_path = "/kaggle/input/machine-learning-hackathon-ii-by-doceree-media/DOCREE_FILES_FINALE_28thJULY/DOCREE_SAMPLE_SUBMISSION.csv"

In [5]:
df_train = pd.read_csv(train_path, encoding='latin-1')
df_test = pd.read_csv(test_path, encoding='latin-1')
df_sub = pd.read_csv(sub_path, encoding='latin-1')

In [6]:
print(df_train.shape)
print(df_test.shape)

(113937, 14)
(28493, 12)


In [7]:
df_train.head(5)

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ID               113937 non-null  int64  
 1   DEVICETYPE       113937 non-null  object 
 2   PLATFORM_ID      113937 non-null  int64  
 3   BIDREQUESTIP     113937 non-null  object 
 4   USERPLATFORMUID  113933 non-null  object 
 5   USERCITY         107578 non-null  object 
 6   USERZIPCODE      109345 non-null  float64
 7   USERAGENT        113935 non-null  object 
 8   PLATFORMTYPE     113937 non-null  object 
 9   CHANNELTYPE      113937 non-null  object 
 10  URL              113937 non-null  object 
 11  KEYWORDS         113937 non-null  object 
 12  TAXONOMY         32313 non-null   object 
 13  IS_HCP           113936 non-null  float64
dtypes: float64(2), int64(2), object(10)
memory usage: 12.2+ MB


In [9]:
df_test.head(5)

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...


In [10]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28493 entries, 0 to 28492
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               28493 non-null  int64  
 1   DEVICETYPE       28493 non-null  object 
 2   PLATFORM_ID      28493 non-null  int64  
 3   BIDREQUESTIP     28493 non-null  object 
 4   USERPLATFORMUID  28493 non-null  object 
 5   USERCITY         26934 non-null  object 
 6   USERZIPCODE      27378 non-null  float64
 7   USERAGENT        28492 non-null  object 
 8   PLATFORMTYPE     28493 non-null  object 
 9   CHANNELTYPE      28493 non-null  object 
 10  URL              28493 non-null  object 
 11  KEYWORDS         28493 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 2.6+ MB


In [11]:
def concatText(x):
    x = x.split('|')
    x = ' '.join(x)
    return x

In [12]:
df_train['KEYWORDS'] = df_train['KEYWORDS'].apply(concatText)
df_test['KEYWORDS'] = df_test['KEYWORDS'].apply(concatText)

In [13]:
df_train['USERAGENT'] = df_train['USERAGENT'].fillna('') 
df_test['USERAGENT'] = df_test['USERAGENT'].fillna('') 

In [14]:
df_train['Total'] = df_train['USERAGENT'] + ' ' + df_train['KEYWORDS'] + ' ' +df_train['URL'] 
df_test['Total'] = df_test['USERAGENT'] + ' ' + df_test['KEYWORDS'] +' ' + df_test['URL'] 

In [15]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2',device='cuda')
start = time.time()
BATCH_SIZE = 4096
for i in tqdm(range(0,len(df_train),BATCH_SIZE)):
    if i == 0:
        train_embed = model.encode(df_train["Total"][i:i+BATCH_SIZE].values,batch_size=BATCH_SIZE,show_progress_bar=False)
    else:
        tmp = model.encode(df_train["Total"][i:i+BATCH_SIZE].values,batch_size=BATCH_SIZE,show_progress_bar=False)
        train_embed = np.concatenate([train_embed,tmp])
        
np.save("./train_embed_2.npy",train_embed)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

  0%|          | 0/28 [00:00<?, ?it/s]

In [16]:
train_embed.shape

(113937, 384)

In [17]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2',device='cuda')
start = time.time()
BATCH_SIZE = 4096
for i in tqdm(range(0,len(df_test),BATCH_SIZE)):
    if i == 0:
        test_embed = model.encode(df_test["Total"][i:i+BATCH_SIZE].values,batch_size=BATCH_SIZE,show_progress_bar=False)
    else:
        tmp2 = model.encode(df_test["Total"][i:i+BATCH_SIZE].values,batch_size=BATCH_SIZE,show_progress_bar=False)
        test_embed = np.concatenate([test_embed,tmp2])

np.save("./test_embed_2.npy",test_embed)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
test_embed.shape

(28493, 384)

In [19]:
import cuml,cupy,cudf
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [20]:
from sklearn.preprocessing import LabelEncoder
le_TAXONOMY = LabelEncoder()
df_train['TAXONOMY'] = le_TAXONOMY.fit_transform(df_train['TAXONOMY'])

In [21]:
df_train['TAXONOMY'].isnull().sum()

0

In [22]:
df_train['TAXONOMY'].value_counts()

207    81624
149     7930
143     6621
43      3997
50      3190
       ...  
10         1
180        1
20         1
65         1
95         1
Name: TAXONOMY, Length: 208, dtype: int64

In [23]:
from cuml.cluster import KMeans
from cuml.neighbors import KNeighborsClassifier

In [24]:
# Sum_of_squared_distances = []
# K = range(1,50)
# for num_clusters in K :
#     kmeans = KMeans(n_clusters=num_clusters)
#     kmeans.fit(train_embed)
#     Sum_of_squared_distances.append(kmeans.inertia_)
# plt.plot(K,Sum_of_squared_distances,'bx-')
# plt.xlabel('Values of K') 
# plt.ylabel('Sum of squared distances/Inertia') 
# plt.title('Elbow Method For Optimal k')
# plt.figure(figsize=(300,80)) 
# plt.show()

In [25]:
KNN = 50
knn_model = KNeighborsClassifier(n_neighbors=KNN, verbose=True)
knn_model.fit(train_embed, df_train.TAXONOMY.values)
KNeighborsClassifier()
preds = knn_model.predict(test_embed)

In [26]:
# KNN = 50
# BATCH_SIZE = 256
# NUM_BATCHES = test_embed.shape[0]//BATCH_SIZE
# model = NearestNeighbors(n_neighbors=KNN)
# model.fit(train_embed)
# test_pred = []

# start = time.time()
# for i in range(NUM_BATCHES + 1):
#     start = i*BATCH_SIZE
#     end = (i+1)*BATCH_SIZE
    
#     distances, indices = model.kneighbors(test_embed[start:end,])
    
#     for k in range(end-start):
#         IDX = np.where(distances[k,] == np.min(distances[k,]))[0]
#         IDS = indices[k,IDX]
#         ls = df_train.iloc[IDS].TAXONOMY.values
#         test_pred.append(ls)
        
#     if(i % 50 == 0):
#         print("DONE PRODUCTS :", end)
#         print("Time : ",int(time.time()-start),"sec")

### SUBMISSION FILE

In [27]:
df_test.shape

(28493, 13)

In [28]:
test_embed.shape

(28493, 384)

In [29]:
df_train.IS_HCP.value_counts()

0.0    79756
1.0    34180
Name: IS_HCP, dtype: int64

In [30]:
sub = df_test[['ID']]
sub['TAXONOMY'] = le_TAXONOMY.inverse_transform(preds)
sub['TAXONOMY'] = sub['TAXONOMY'].fillna(-1)
sub

/tmp/ipykernel_23/1571234471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['TAXONOMY'] = le_TAXONOMY.inverse_transform(preds)
/tmp/ipykernel_23/1571234471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['TAXONOMY'] = sub['TAXONOMY'].fillna(-1)


,ID,TAXONOMY
0,115501,-1
1,115502,2084P0800X
2,115503,-1
3,115504,-1
4,115505,-1
...,...,...
28488,143989,-1
28489,143990,-1
28490,143991,-1
28491,143992,-1


In [31]:
sub['TAXONOMY'].value_counts()

-1            21611
2084P0800X     3162
2084N0400X     2008
207Q00000X      621
207R00000X      218
207RG0100X      179
207N00000X      118
208000000X      110
2084N0600X       75
2085R0001X       60
207RH0003X       53
207RC0000X       52
207ZP0105X       38
207T00000X       31
207W00000X       28
2084A0401X       17
2084A2900X       17
363A00000X       16
390200000X       16
207P00000X       12
207L00000X       11
2084N0402X        9
2084P2900X        9
207RR0500X        7
207RI0200X        5
207VM0101X        5
363LF0000X        2
2084P0805X        1
103TC0700X        1
207RE0101X        1
Name: TAXONOMY, dtype: int64

In [32]:
sub.to_csv('submission_new.csv',index=False)